In [5]:
import xarray as xr
import numpy as np
from pathlib import Path
import numpy as np
import cftime

In [10]:
out_dir = Path("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/")
out_dir.mkdir(exist_ok=True, parents=True)

# Ens files
ens_dir = Path("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/DATA/")
ens_clitter = xr.open_dataset(ens_dir / "cLitter/ENSMEAN/ENSMEAN_S3_cLitter.nc")["cLitter"]
ens_csoil = xr.open_dataset(ens_dir / "cSoil/ENSMEAN/ENSMEAN_S3_cSoil.nc")["cSoil"]
ens_cveg = xr.open_dataset(ens_dir / "cVeg/ENSMEAN/ENSMEAN_S3_cVeg.nc")["cVeg"]
ens_mrso = xr.open_dataset(ens_dir / "mrso/ENSMEAN/ENSMEAN_S3_mrso.nc")["mrso"]
ens_lai = xr.open_dataset(ens_dir / "lai/ENSMEAN/ENSMEAN_S3_lai.nc")["lai"]
ens_ctotal = xr.open_dataset(ens_dir / "cTotal/ENSMEAN/ENSMEAN_S3_cTotal_monthly.nc")["cTotal"]
ens_das = {"cLitter": ens_clitter,
            "cSoil": ens_csoil,
            "cVeg": ens_cveg,
            "mrso": ens_mrso,
            "lai": ens_lai,
            "cTotal": ens_ctotal}

# Stable files
stable_dir = "/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/MODELS/Stable-Emulator"
stable_lai = xr.open_dataset(f"{stable_dir}/lai.nc")["lai"]
stable_cLitter = xr.open_dataset(f"{stable_dir}/cLitter.nc")["cLitter"]
stable_cSoil = xr.open_dataset(f"{stable_dir}/cSoil.nc")["cSoil"]
stable_cVeg = xr.open_dataset(f"{stable_dir}/cVeg.nc")["cVeg"]
stable_mrso = xr.open_dataset(f"{stable_dir}/mrso.nc")["mrso"]
stable_cTotal = xr.open_dataset(f"{stable_dir}/cTotal.nc")["cTotal"]
stables_das = {"cLitter": stable_cLitter,
                "cSoil": stable_cSoil,
                "cVeg": stable_cVeg,
                "mrso": stable_mrso,    
                "lai": stable_lai,
                "cTotal": stable_cTotal}

# Base files
base_dir = Path("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/MODELS/Base-Emulator")
base_lai = xr.open_dataset(f"{base_dir}/S3_lai.nc")["lai"]
base_cLitter = xr.open_dataset(f"{base_dir}/S3_cLitter.nc")["cLitter"]
base_cSoil = xr.open_dataset(f"{base_dir}/S3_cSoil.nc")["cSoil"]
base_cVeg = xr.open_dataset(f"{base_dir}/S3_cVeg.nc")["cVeg"]
base_mrso = xr.open_dataset(f"{base_dir}/S3_mrso.nc")["mrso"]
base_cTotal = xr.open_dataset(f"{base_dir}/S3_cTotal.nc")["cTotal"]
base_das = {"cLitter": base_cLitter,
             "cSoil": base_cSoil,
             "cVeg": base_cVeg,
             "mrso": base_mrso,
             "lai": base_lai,
             "cTotal": base_cTotal}

# tvt
tvt_mask = xr.open_dataset("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/data/masks/tvt_mask.nc")["tvt_mask"]

FileNotFoundError: [Errno 2] No such file or directory: '/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/MODELS/Stable-Emulator/lai.nc'

In [7]:
masked_ens = {}
masked_stable = {}
masked_base = {}

for var, da in ens_das.items():
    da_masked = da.where(tvt_mask.isin([0, 1, 2]))
    masked_ens[var] = da_masked
for var, da in stables_das.items():
    da_masked = da.where(tvt_mask.isin([0, 1, 2]))
    masked_stable[var] = da_masked
for var, da in base_das.items():
    da_masked = da.where(tvt_mask.isin([0, 1, 2]))
    masked_base[var] = da_masked

In [8]:
start_year = 1901
end_year   = 2023

for var, da in masked_base.items():
    if var in ['mrso', 'lai','cTotal']:
        out_path = out_dir / f"BASE_S3_{var}.nc"
        da_m.to_netcdf(out_path)
        print(f"Saved monthly {var} to {out_path}")
    else:
        # --- repeat each annual value 12× along time ---
        n_years = da.sizes["time"]
        rep_idx = np.repeat(np.arange(n_years), 12)
        da_m = da.isel(time=rep_idx)

        # --- build new monthly time axis 1901-01 to 2023-12 ---
        years = np.repeat(np.arange(start_year, end_year + 1), 12)
        months = np.tile(np.arange(1, 13), end_year - start_year + 1)

        if len(years) != da_m.sizes["time"]:
            raise ValueError(
                f"Length mismatch: have {da_m.sizes['time']} time steps, "
                f"but built {len(years)} monthly timestamps."
            )

        new_time = [
            cftime.DatetimeNoLeap(int(y), int(m), 1)
            for y, m in zip(years, months)
        ]

        da_m = da_m.assign_coords(time=("time", new_time))

        # keep CF-style metadata consistent
        da_m.time.encoding.update({
            "units": "days since 1901-01-01 00:00:00",
            "calendar": "noleap",
        })

        # save
        out_path = out_dir / f"BASE_S3_{var}.nc"
        da_m.to_netcdf(out_path)
        print(f"Saved monthly {var} to {out_path}")

Saved monthly cLitter to /Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/BASE_S3_cLitter.nc
Saved monthly cSoil to /Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/BASE_S3_cSoil.nc
Saved monthly cVeg to /Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/BASE_S3_cVeg.nc
Saved monthly mrso to /Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/BASE_S3_mrso.nc
Saved monthly lai to /Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/BASE_S3_lai.nc
Saved monthly cTotal to /Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisat

In [9]:
for var, da in masked_stable.items():

    out_path = out_dir / f"STABLE_S3_{var}.nc"
    da.to_netcdf(out_path)

for var, da in masked_ens.items():
    
    out_path = out_dir / f"ENSMEAN_S3_{var}.nc"
    da.to_netcdf(out_path)